In [0]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, GRU, Dropout, Embedding
from music21 import converter, instrument, note, chord, pitch
from google.colab import drive
import numpy as np
import time
import glob
import pickle

In [65]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
data_path = "/content/gdrive/My Drive/Applied Deep Learning/Data/"

In [0]:
def get_notes(filename):
  notes = []
  midi = converter.parse(filename)
  notes_to_parse = None
  try:  # file has instrument parts
    s2 = instrument.partitionByInstrument(midi)
    notes_to_parse = s2.parts[0].recurse() 
  except:  # file has notes in a flat structure
    notes_to_parse = midi.flat.notes

  for element in notes_to_parse:
    
    new_chord = [-1] * 6 # 5 pitches + 1 duration
    
    if isinstance(element, note.Note):
      new_chord[0] = element.pitch.ps
      new_chord[5] = float(element.duration.quarterLength)
      notes.append(new_chord)
    elif isinstance(element, chord.Chord):
      new_chord[5] = float(element.duration.quarterLength)
      
      for k in range(len(element.pitches)):
        new_chord[k] = element.pitches[k].ps
        
        if k >= len(new_chord) - 1:
          break

      notes.append(new_chord)

  return notes


## Load trained outputs

In [0]:
preprocessed_path = "/content/gdrive/My Drive/Applied Deep Learning/Outputs/"

In [0]:
# new_songs_game100, chopin, game

dataset = 'new_songs_game100' 

In [0]:
all_songs_data = np.load(preprocessed_path + '{}.npy'.format(dataset))

In [74]:
all_songs_data.shape

(100, 2500, 6)

## Generate MIDI


In [0]:
from music21 import instrument, note, stream, chord

In [0]:
def create_midi(prediction_output, number, output_path, dataset):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        duration = pattern[-1]
        
        if int(duration) != -1:
            notes_in_chord = pattern[:-1]
            notes = []
            for current_note in notes_in_chord:
                if int(current_note) != -1:
                    new_note = note.Note(int(current_note))
                    new_note.storedInstrument = instrument.Piano()
                    notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)    

            offset += duration

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp=output_path+'{}_output{}.mid'.format(dataset, number))

In [0]:
output_path = "/content/gdrive/My Drive/Applied Deep Learning/{} Output/".format(dataset)

In [0]:
import os
os.mkdir(output_path)

In [0]:
for k in range(all_songs_data.shape[0]):
  create_midi(all_songs_data[k], k, output_path, dataset)